<center><font color=#555555><font size=6> MAF1731 - Microstructure and Trading Systems </font> <br> <br>
    
<center><font color=#555555><font size=5> B.Eng Financial Engineering </font> <br> <br> <br> <br>
   
<center> <font color= #555555> <font size = 4> May 2023 </a> | <font color= #555555> <font size = 4> Repository: <a href='https://github.com/CarlosCuevasM'>Link</a></font>

<hr style="border:0.02in solid gray"> </hr>

In [71]:
# -- Generic
import pandas as pd
import numpy as np
import json


In [72]:
# -- Project scripts
import visualizations as vs
import functions as fn
import data as dt

In [73]:
# Opening JSON file
f = open('files/orderbooks_05jul21.json')
# Returns JSON object as a dictionary
orderbooks_data = json.load(f)

In [74]:
orderbooks_data.keys()

dict_keys(['bitfinex', 'kraken'])

In [75]:
# ----- Market information
df_bitfinex_tob = pd.DataFrame(orderbooks_data['bitfinex']).transpose().reset_index()
df_bitfinex_tob['index'] = pd.to_datetime(df_bitfinex_tob['index'])
df_bitfinex_tob = df_bitfinex_tob.set_index('index').resample('min').last().ffill()
for column in df_bitfinex_tob.columns:
    df_bitfinex_tob[column] = df_bitfinex_tob[column].apply(lambda x: x['0'])
df_bitfinex_tob = df_bitfinex_tob.reset_index().rename(columns = {'index':'timestamp'})


In [76]:
df_bitfinex_tob

,timestamp,ask_size,ask,bid,bid_size
0,2021-07-05 13:06:00+00:00,0.100600,28283.0,28282.0,0.338172
1,2021-07-05 13:07:00+00:00,2.792849,28325.0,28323.0,0.594797
2,2021-07-05 13:08:00+00:00,0.403360,28315.0,28309.0,0.093481
3,2021-07-05 13:09:00+00:00,0.148733,28333.0,28329.0,0.339650
4,2021-07-05 13:10:00+00:00,0.003524,28341.0,28336.0,3.807731
...,...,...,...,...,...
56,2021-07-05 14:02:00+00:00,3.550600,28292.0,28288.0,0.446627
57,2021-07-05 14:03:00+00:00,0.385010,28301.0,28295.0,0.600127
58,2021-07-05 14:04:00+00:00,0.009931,28352.0,28340.0,0.357682
59,2021-07-05 14:05:00+00:00,0.445649,28361.0,28355.0,0.006000


In [77]:
#df_orders
timestamp= [df_bitfinex_tob['timestamp'][0],df_bitfinex_tob['timestamp'][0]]
order_id=[1,2]
side=['buy','sell']
price=[df_bitfinex_tob['ask'][0],df_bitfinex_tob['bid'][0]]
order_amount=[0.0001,0.0001]
symbol=['BTC/USDT','BTC/USDT']





In [78]:
for i in range(1,len(df_bitfinex_tob)):

    if df_bitfinex_tob['ask'][i]>df_bitfinex_tob['ask'][i-1]:
        order_id.append(i+2)
        timestamp.append(df_bitfinex_tob['timestamp'][i])
        side.append('buy')
        price.append(df_bitfinex_tob['ask'][i])
        order_amount.append(0.0001)
        symbol.append('BTC/USDT')
        if df_bitfinex_tob['ask_size'][i]<0.0001:
            order_amount.append(df_bitfinex_tob['ask_size'][i])

    if df_bitfinex_tob['bid'][i]<df_bitfinex_tob['bid'][i-1]:
        order_id.append(i+2)
        timestamp.append(df_bitfinex_tob['timestamp'][i])
        side.append('sell')
        price.append(df_bitfinex_tob['bid'][i])
        order_amount.append(0.0001)
        symbol.append('BTC/USDT')
        if df_bitfinex_tob['bid_size'][i]<0.0001:
            order_amount.append(df_bitfinex_tob['bid_size'][i])



In [79]:
df_orders=pd.DataFrame({'timestamp':timestamp,
              'order_id':order_id,
              'side':side,
              'price':price,
              'order_amount':order_amount,
              'symbol':symbol})
df_orders

,timestamp,order_id,side,price,order_amount,symbol
0,2021-07-05 13:06:00+00:00,1,buy,28283.0,0.0001,BTC/USDT
1,2021-07-05 13:06:00+00:00,2,sell,28282.0,0.0001,BTC/USDT
2,2021-07-05 13:07:00+00:00,3,buy,28325.0,0.0001,BTC/USDT
3,2021-07-05 13:08:00+00:00,4,sell,28309.0,0.0001,BTC/USDT
4,2021-07-05 13:09:00+00:00,5,buy,28333.0,0.0001,BTC/USDT
...,...,...,...,...,...,...
57,2021-07-05 14:02:00+00:00,58,buy,28292.0,0.0001,BTC/USDT
58,2021-07-05 14:03:00+00:00,59,buy,28301.0,0.0001,BTC/USDT
59,2021-07-05 14:04:00+00:00,60,buy,28352.0,0.0001,BTC/USDT
60,2021-07-05 14:05:00+00:00,61,buy,28361.0,0.0001,BTC/USDT


In [80]:
#df_trades
timestamps_tr=[]
order_id_tr=[]
trade_id_tr=[]
side_tr=[]
price_tr=[]
symbol_tr=[]
filled_amount_tr=[]
order_amount_tr=[]
fee_tr=[]


In [81]:
df_bitfinex_tob.drop(0)
for i in range(len(df_bitfinex_tob)):
    trade_id_tr.append(i)
    
    if side[i]=='buy':
        for x in range (i,len(df_bitfinex_tob)):

            if price[i]<df_bitfinex_tob['ask'][x]:
                timestamps_tr.append(df_bitfinex_tob['timestamp'][x])
                order_id_tr.append(order_id[i])
                side_tr.append(side[i])
                price_tr.append(price[i])
                symbol_tr.append('BTC/USDT')
                order_amount_tr.append(order_amount[i])
                fee_tr.append(price[i]*0.0025*0.0001)
                if (order_amount[i]-df_bitfinex_tob['ask_size'][x])>0:
                    filled_amount_tr.append(order_amount[i]-df_bitfinex_tob['ask_size'][x])
                    break             
                else:
                    filled_amount_tr.append(order_amount[i])
                    break

    if side[i]=='sell':
        for y in range (i,len(df_bitfinex_tob)):

            if price[i]<df_bitfinex_tob['bid'][y]:
                timestamps_tr.append(df_bitfinex_tob['timestamp'][y])
                order_id_tr.append(order_id[i])
                side_tr.append(side[i])
                price_tr.append(price[i])
                symbol_tr.append('BTC/USDT')
                order_amount_tr.append(order_amount[i])
                fee_tr.append(price[i]*0.0025*0.0001)
                if (order_amount[i]-df_bitfinex_tob['bid_size'][y])>0:
                    filled_amount_tr.append(order_amount[i]-df_bitfinex_tob['bid_size'][y])  
                    break
                else:
                    filled_amount_tr.append(order_amount[i])
                    break

In [82]:
df_trades=pd.DataFrame({'timestamp':timestamps_tr,
              'order_id':order_id_tr,
              #'trade_id':trade_id_tr,
              'side':side_tr,
              'price':price_tr,
              'symbol':symbol_tr,
              'filled_amount':filled_amount_tr,
              'order_amount':order_amount_tr,
              'fee':fee_tr
            })
df_trades.head()

,timestamp,order_id,side,price,symbol,filled_amount,order_amount,fee
0,2021-07-05 13:07:00+00:00,1,buy,28283.0,BTC/USDT,0.0001,0.0001,0.007071
1,2021-07-05 13:07:00+00:00,2,sell,28282.0,BTC/USDT,0.0001,0.0001,0.007071
2,2021-07-05 13:09:00+00:00,3,buy,28325.0,BTC/USDT,0.0001,0.0001,0.007081
3,2021-07-05 13:09:00+00:00,4,sell,28309.0,BTC/USDT,0.0001,0.0001,0.007077
4,2021-07-05 13:10:00+00:00,5,buy,28333.0,BTC/USDT,0.0001,0.0001,0.007083


In [83]:
#df_inventory

timestamp_inv=[df_orders['timestamp'][0]]
base=[5]
quote=[500000]


In [84]:
for i in range(len(df_trades)):
    
    if df_trades['side'][i]=='buy':
        timestamp_inv.append(df_trades['timestamp'][i])
        base.append(base[i]+df_trades['filled_amount'][i])
        quote.append(quote[i]-df_trades['fee'][i]-(df_trades['price'][i]*df_trades['filled_amount'][i]))



    if df_trades['side'][i]=='sell':
        timestamp_inv.append(df_trades['timestamp'][i])
        base.append(base[i]-df_trades['filled_amount'][i])
        quote.append(quote[i]-df_trades['fee'][i]+(df_trades['price'][i]*df_trades['filled_amount'][i]))
     



In [85]:
df_inventory=pd.DataFrame({'timestamp':timestamp_inv,
                           'base':base,
                           'quote':quote})

df_inventory

,timestamp,base,quote
0,2021-07-05 13:06:00+00:00,5.0000,500000.000000
1,2021-07-05 13:07:00+00:00,5.0001,499997.164629
2,2021-07-05 13:07:00+00:00,5.0000,499999.985759
3,2021-07-05 13:09:00+00:00,5.0001,499997.146177
4,2021-07-05 13:09:00+00:00,5.0000,499999.970000
5,2021-07-05 13:10:00+00:00,5.0001,499997.129617
6,2021-07-05 13:11:00+00:00,5.0002,499994.288432
7,2021-07-05 13:12:00+00:00,5.0003,499991.446545
8,2021-07-05 13:45:00+00:00,5.0004,499988.599946
9,2021-07-05 13:41:00+00:00,5.0003,499991.431051
